<a href="https://colab.research.google.com/github/Sriayuningsih02/Deep-Learning-Lanjut/blob/main/Deep_Learning_Lanjut_Tugas_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
import time
import functools

# --- KONFIGURASI FILE ---
# Pastikan nama file ini sesuai dengan yang Anda upload di Colab
content_path = 'ikan.jpg'
style_path = 'batik.jpg'

print("TensorFlow Version:", tf.__version__)
print("GPU Available:", len(tf.config.list_physical_devices('GPU')) > 0)

# --- FUNGSI UTILITAS GAMBAR ---

def tensor_to_image(tensor):
  """Mengubah tensor kembali menjadi gambar untuk ditampilkan"""
  tensor = tensor * 255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor) > 3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

def load_img(path_to_img):
  """Memuat gambar dan mengubah ukurannya agar tidak terlalu besar untuk Colab"""
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)
  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)
  plt.imshow(image)
  if title:
    plt.title(title)

# --- MEMUAT GAMBAR ---
content_image = load_img(content_path)
style_image = load_img(style_path)

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
imshow(content_image, 'Content Image (Ikan)')
plt.subplot(1, 2, 2)
imshow(style_image, 'Style Image (Batik)')
plt.show()

# --- DEFINISI MODEL VGG19 ---

# Kita menggunakan VGG19 pretrained untuk ekstraksi fitur
# Content layer: Layer yang lebih dalam menangkap bentuk objek
content_layers = ['block5_conv2']

# Style layers: Layer awal hingga menengah menangkap tekstur dan pola
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

def vgg_layers(layer_names):
  """ Membuat model VGG19 yang mengembalikan output dari layer perantara """
  vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False

  outputs = [vgg.get_layer(name).output for name in layer_names]
  model = tf.keras.Model([vgg.input], outputs)
  return model

class StyleContentModel(tf.keras.models.Model):
  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.vgg = vgg_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.vgg.trainable = False

  def call(self, inputs):
    "Mengharapkan input float dalam range [0,1]"
    inputs = inputs * 255.0
    # Preprocessing standar VGG19
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers],
                                      outputs[self.num_style_layers:])

    # Menghitung Gram Matrix untuk Style
    style_outputs = [gram_matrix(style_output)
                     for style_output in style_outputs]

    content_dict = {content_name: value
                    for content_name, value
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name: value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}

    return {'content': content_dict, 'style': style_dict}

def gram_matrix(input_tensor):
  """ Menghitung Gram Matrix (korelasi antar filter) untuk representasi gaya """
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)

# --- EKSTRAKTOR FITUR ---
extractor = StyleContentModel(style_layers, content_layers)

# Target Style dan Content
style_targets = extractor(style_image)['style']
content_targets = extractor(content_image)['content']

# --- OPTIMISASI ---

# Gambar awal adalah copy dari content image (bisa juga noise acak)
image = tf.Variable(content_image)

def clip_0_1(image):
  """Menjaga nilai pixel tetap di range 0-1"""
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

# Optimizer
opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

# Bobot Loss (Penting untuk mengatur keseimbangan)
style_weight = 1e-2  # Tingkatkan jika ingin lebih mirip batik
content_weight = 1e4 # Tingkatkan jika bentuk ikan hilang
total_variation_weight = 30 # Untuk menghaluskan gambar (mengurangi noise)

def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']

    # Loss Style
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2)
                           for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    # Loss Content
    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2)
                             for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers

    loss = style_loss + content_loss
    return loss

@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)
    # Tambahkan Total Variation Loss untuk hasil lebih halus
    loss += total_variation_weight * tf.image.total_variation(image)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

# --- TRAINING LOOP ---

print("Mulai Training Neural Style Transfer...")
start = time.time()

epochs = 10
steps_per_epoch = 100

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end='', flush=True)

  # Tampilkan hasil setiap epoch
  print(f"\nEpoch: {n+1}/{epochs}")
  plt.figure(figsize=(8,8))
  imshow(image, title=f'Epoch {n+1}')
  plt.axis('off')
  plt.show()

end = time.time()
print(f"Total waktu: {end-start:.1f} detik")

# Simpan hasil akhir
final_img = tensor_to_image(image)
final_img.save('ikan_batik_style.jpg')
print("Gambar disimpan sebagai 'ikan_batik_style.jpg'")